In [ ]:
# imports
import pandas as pd
import numpy as np

# show multiple outputs from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Read in Vegas Odds Data

In [ ]:
combined_odds_df = pd.DataFrame()

In [ ]:
# adjust year to get all excel files
# year = '2009-10'
# year = '2010-11'
# year = '2011-12'
# year = '2012-13'
# year = '2013-14'
# year = '2014-15'
# year = '2015-16'
# year = '2016-17'
# year = '2017-18'
# year = '2018-19'
year = '2019-20'

In [ ]:
# read from local drive with excel files
odds_year = pd.read_excel('nfl odds ' + year + '.xlsx')

In [ ]:
def to_date(dt):
    d = str(dt)[-2:]
    m = str(dt)[:-2]
    y = year.split('-')[0]
    if int(m) < 3:
        y = str(int(y) + 1)
        m = '0' + m
    date = y + m + d
    return pd.to_datetime(date, format='%Y%m%d')

In [ ]:
odds_year['Date'] = odds_year['Date'].apply(to_date)

In [ ]:
def to_abbr(tm):
    team_dict = {
        'GreenBay' : 'GB',
        'Chicago' : 'CHI',
        'Atlanta' : 'ATL',
        'Minnesota' : 'MIN',
        'Washington' : 'WAS',
        'Philadelphia' : 'PHI',
        'Buffalo' : 'BUF',
        'BuffaloBills' : 'BUF',
        'NYJets' : 'NYJ',
        'NewYork' : 'NYG',
        'Baltimore' : 'BAL',
        'Miami' : 'MIA',
        'SanFrancisco' : 'SF',
        'TampaBay' : 'TB',
        'KansasCity' : 'KC',
        'Jacksonville' : 'JAX',
        'Tennessee' : 'TEN',
        'Cleveland' : 'CLE',
        'LARams' : 'LA',
        'LosAngeles' : 'LA',
        'St.Louis' : 'STL',
        'Carolina' : 'CAR',
        'Detroit' : 'DET',
        'Arizona' : 'ARI',
        'Cincinnati' : 'CIN',
        'Seattle' : 'SEA',
        'Indianapolis' : 'IND',
        'LAChargers' : 'LAC',
        'SanDiego' : 'SD',
        'NYGiants' : 'NYG',
        'Dallas' : 'DAL',
        'Pittsburgh' : 'PIT',
        'NewEngland' : 'NE',
        'Houston' : 'HOU',
        'NewOrleans' : 'NO',
        'Denver' : 'DEN',
        'Oakland' : 'OAK'
    }
    return team_dict[tm]

In [ ]:
odds_year['Team'] = odds_year['Team'].apply(to_abbr)

In [ ]:
odds_year.loc[(odds_year['Close'] == 'pk'), 'Close'] = 0

In [ ]:
odds_year[['Close', 'ML']] = odds_year[['Close', 'ML']].astype(float)

#### Create win probabilty and projected points fields

In [ ]:
def setVorH(row):
    if row.name % 2 == 0:
        return 'V'
    else:
        return 'H'

odds_year['VH'] = odds_year.apply(setVorH, axis=1)

In [ ]:
pr_pts = []
wps = []
for idx, row in odds_year.iterrows():
    if row['ML'] > 0:
        total = row['Close']
        if row['VH'] == 'V':
            spd = odds_year.iloc[idx+1]['Close']
        else:
            spd = odds_year.iloc[idx-1]['Close']
        proj_pts = (total - spd) / 2
        win_pb = 100 / (row['ML'] + 100)
    else:
        spd = row['Close']
        if row['VH'] == 'V':
            total = odds_year.iloc[idx+1]['Close']
        else:
            total = odds_year.iloc[idx-1]['Close']
        proj_pts = (total + spd) / 2
        win_pb = -row['ML'] / (100 - row['ML'])

    pr_pts.append(proj_pts)
    wps.append(win_pb)

odds_year['Projected Pts'] = pr_pts
odds_year['Win Prob'] = wps

In [ ]:
# append all yearly dataframes
combined_odds_df = combined_odds_df.append(odds_year, ignore_index=True)

In [ ]:
combined_odds_df['Date'] = pd.to_datetime(combined_odds_df['Date'])

In [ ]:
combined_odds_df.isna().sum()

In [ ]:
consolidated_output = pd.read_csv('ConsolidateOutput.csv')

In [ ]:
consolidated_output['game_date'] = pd.to_datetime(consolidated_output['game_date'])

In [ ]:
combined_df = pd.merge_asof(consolidated_output, combined_odds_df[['Date', 'Team', 'Projected Pts', 'Win Prob']], left_on='game_date', right_on='Date', left_by='posteam', right_by='Team', tolerance=pd.Timedelta('3days'), direction='nearest')

In [ ]:
x = combined_df[combined_df['posteam'].isna() == False]
x[['Win Prob', 'Projected Pts']].isna().sum()

In [ ]:
combined_df.to_csv('ConsolidateOutput_plus_WP.csv')

### Create Adjusted Win Probability and Point Scored vs Pace Metrics

In [ ]:
pbp = combined_df

In [ ]:
condensed_pbp = pbp[['play_id', 'game_id', 'posteam', 'defteam', 'yardline_100', 'game_seconds_remaining', 'drive', 'down', 'goal_to_go', 'ydstogo', 'ydsnet', 'play_type', 'yards_gained', 'shotgun', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'run_location', 'run_gap', 'posteam_timeouts_remaining','defteam_timeouts_remaining', 'posteam_score', 'defteam_score', 'score_differential', 'ep', 'epa', 'air_epa', 'yac_epa', 'comp_air_epa', 'comp_yac_epa', 'wp', 'wpa', 'air_wpa', 'comp_yac_wpa', 'incomplete_pass', 'interception', 'fumble_forced', 'penalty', 'qb_hit', 'sack', 'touchdown', 'complete_pass', 'penalty_team', 'penalty_yards', 'Projected Pts', 'Win Prob']]

In [ ]:
condensed_pbp.dropna(subset=['posteam', 'posteam_score'], inplace=True)
condensed_pbp['wp'] = condensed_pbp['wp'].fillna(method='bfill')
condensed_pbp['wpa'] = condensed_pbp['wpa'].fillna(0)

In [ ]:
condensed_pbp.isna().sum()

In [ ]:
game_breakdown = condensed_pbp.groupby('game_id')

#### Breakdown by game in order to calculate metrics

In [ ]:
game_df_dict = {}

for name, game in game_breakdown:
    game['pt_pace'] = (3600 - game['game_seconds_remaining']) / 3600 * game['Projected Pts']
    game['pts_scored_vs_pace'] = game['posteam_score'] - game['pt_pace']
    game = game.reset_index()
    game_df_dict[name] = game.copy(deep=True)

In [ ]:
# iterate by team by game
for game in game_df_dict.values():
    prev_team = game.loc[0, 'posteam']

    for idx, row in game.iterrows():
        if idx == 0:
            game['adj_wp'] = game['Win Prob']
            game['def_team_pts_scored_vs_pace'] = game['pts_scored_vs_pace']
            continue

        if row['posteam'] != prev_team:
            wpa = game.loc[idx-1, 'wpa']
        else:
            wpa = row['wp'] - game.loc[idx-1, 'wp']

        if game.loc[idx-1, 'wp'] >= .99 or game.loc[idx-1, 'wp'] <= .01:
            modifier = 0
        else:
            modifier = 1
            if game.loc[idx-1, 'adj_wp'] >= game.loc[idx-1, 'wp']:
                if wpa > 0:
                    modifier = 1 - (((game.loc[idx-1, 'adj_wp'] - game.loc[idx-1, 'wp']) / (game.loc[idx-1, 'wp'])) * (row['game_seconds_remaining'] / 3600))
            else:
                if wpa < 0:
                    modifier = 1 - (((game.loc[idx-1, 'wp'] - game.loc[idx-1, 'adj_wp']) / (game.loc[idx-1, 'wp'])) * (row['game_seconds_remaining'] / 3600))

        adj_wp = (game.loc[idx-1, 'adj_wp'] + (wpa * modifier))\

        if row['posteam'] != prev_team:
            game.loc[idx, 'adj_wp'] = 1 - adj_wp
            game.loc[idx, 'def_team_pts_scored_vs_pace'] = game.loc[idx - 1, 'pts_scored_vs_pace']
        else:
            game.loc[idx, 'adj_wp'] = adj_wp
            game.loc[idx, 'def_team_pts_scored_vs_pace'] = game.loc[idx - 1, 'def_team_pts_scored_vs_pace']

        prev_team = row['posteam']

In [ ]:
new_features = pd.DataFrame()

for game in game_df_dict.values():
    relevant_df = game[['play_id', 'game_id', 'pts_scored_vs_pace', 'def_team_pts_scored_vs_pace', 'adj_wp']]
    new_features = new_features.append(relevant_df, ignore_index=True)

In [ ]:
# add new features to dataset
new_features.to_csv('adjWP_PTpace.csv')
